In [1]:
################### Downloading and some initial processing ######################
# (ٔNote: more advanced processing is done in another file)

"""
The code has been made by a group of Machine Learning in Physics course, in 2021.
Member of the group: hamid Pour Mohammad, Saeid Entezari, Mahdi Naghi Lou (and Reza Bakhoda)
Extraction of data happens in 3 cells here; first, second and third cells.
Please note that you may see some errors when you are collecting the data from twitter--because of VPN limits,
twitter database, internet speed, etc. Thus, you may need to collect them by part.

"""

import tweepy
import re
import csv
import pandas as pd
from numpy import array, add, zeros, asmatrix, savetxt
from nltk.stem import PorterStemmer 
from collections import Counter

# input your credentials here: please note that everyone has special credentials, for the special purpose.
access_token=""
access_token_secret=""
consumer_key=""
consumer_secret=""

# building a portal:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

alp=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
     'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ') # english alphabet, and the space

def true_text(text): # making sure that each character is in 'alp'
    rebuild=''
    for t in text:
        if t in alp: rebuild+=t
    return(rebuild)

def connector(df): # connecting the items of the dataframe
    connected=[]
    for i in df:
        connected = connected + i
    return(connected)
   
ps = PorterStemmer() # to make each word in a 'stem' condition; it helps us to find the root of each word

most_words_list=[] # most words used in all classes

nt=4000 # number of tweets that you want to be collected for each tagword.

# reading the file of tags (external csv file):
tags=pd.read_csv('tags.csv')
tag_list=array(tags['the_tags'])

# Making a frame, that will contain some lists. Each list is for one tag. We will put all these lists together.
frames=[]

In [2]:
# Creating the first iteration: (it takes about two minutes)
# We want to build a 'tagword', and its related 'tagword-filter:retweets';
# because a lot of tweets are repetitious.
word='#'+tag_list[0] # here is just the 'tagword'
new_search=word
new_search+="-filter:retweets" # here is the 'tagword-filter:retweets'
print(f'IN {word}...') # it shows in which tagword we are.

# Collecting the data for the first tagword, with a generator:
tweets=tweepy.Cursor(api.search, q=new_search, count=100, 
                           lang="en",
                           since="2021-03-01").items(nt)

# Separating the features of each tweet:
tweet_information = [[tweet.id, tweet.user.screen_name, tweet.created_at, tweet.text] 
                     for tweet in tweets]

# Creating a Pandas Dataframe from 'tweet_information' data:
tweet_information = pd.DataFrame(data=tweet_information, 
                    columns=['id', 'user', "created_at", 'text'])

# cleaning the tweets:
tweet_information['text']=tweet_information['text'].str.lower() # convert text to lowercase
tweet_information['text']=tweet_information['text'].replace('#', ' ').replace('\n', ' ') # remove # and \n
tweet_information['text']=tweet_information['text'].apply(lambda text: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split()))
tweet_information['text']=tweet_information['text'].replace('_', ' ')  # eliminate the _ sign
tweet_information['text']=tweet_information['text'].apply(lambda text: true_text(text)) # just to make sure that the tweet is a text
tweet_information['text']=tweet_information['text'].apply(lambda text: text.split()) # splitting the text
tweet_information['text']=tweet_information['text'].apply(lambda text: list(ps.stem(word) for word in text)) # to use the base form of the words

most_words = tweet_information['text'] # most used words for this tagword
most_words = connector(most_words) # connecting the items of this dataframe
most_words = Counter(" ".join(most_words).split()).most_common(int(nt/15))
most_words = pd.DataFrame(most_words, columns=['Word', 'Frequency'])
most_words_list = most_words_list + list(most_words['Word']) # add to most used words list

tweet_information['text_lenght']= tweet_information['text'].apply(lambda text: len(text))
tweet_information['tag']=true_text(word) # writing the related 'tagword'
tweet_information['built_in_tag']=new_search # writing the related 'tagword-filter:retweets'
 
frames.append(tweet_information) # appending this tweet_information to 'frames' list

tweet_information.head()

IN love...


,id,user,created_at,text,text_lenght,tag,built_in_tag
0,1371947094085402628,AiCuddles,2021-03-16 22:10:51,"['hi', 'what', 'did', 'i', 'do', 'wrong', 'lov...",10,love,#love-filter:retweets
1,1371946904012140545,todaygame,2021-03-16 22:10:06,"['medicalintuit', 'healer', 'interview', 'auth...",9,love,#love-filter:retweets
2,1371946895174737924,CSCarrigan,2021-03-16 22:10:04,"['medicalintuit', 'healer', 'interview', 'auth...",9,love,#love-filter:retweets
3,1371946895157919748,UkHealing,2021-03-16 22:10:04,"['medicalintuit', 'healer', 'interview', 'auth...",9,love,#love-filter:retweets
4,1371946852464091136,JanderlynWMusic,2021-03-16 22:09:53,"['thank', 'you', 'lord', 'for', 'all', 'you', ...",17,love,#love-filter:retweets


In [3]:
# NOW, let's do it for all 'tagword's:

# (Note: it takes about 2 hours)

# We did this iteration for the first tagword, so we don't need to do it again; just others:
tag_list=tag_list[1:] 

for i in tag_list:
    word='#'+i
    new_search=word
    new_search+="-filter:retweets"
    print(f'IN {word}...')

    tweets=tweepy.Cursor(api.search, q=new_search, count=100, 
                               lang="en",
                               since="2021-03-01").items(nt)
    
    tweet_information = [[tweet.id, tweet.user.screen_name, tweet.created_at, tweet.text] 
                         for tweet in tweets]

    tweet_information = pd.DataFrame(data=tweet_information, 
                        columns=['id', 'user', "created_at", 'text'])

    tweet_information['text']=tweet_information['text'].str.lower()
    tweet_information['text']=tweet_information['text'].replace('#', ' ').replace('\n', ' ')    
    tweet_information['text']=tweet_information['text'].apply(lambda text: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split()))
    tweet_information['text']=tweet_information['text'].replace('_', ' ')  
    tweet_information['text']=tweet_information['text'].apply(lambda text: true_text(text))
    tweet_information['text']=tweet_information['text'].apply(lambda text: text.split())
    tweet_information['text']=tweet_information['text'].apply(lambda text: list(ps.stem(word) for word in text))
    
    most_words = tweet_information['text']
    most_words = connector(most_words)
    most_words = Counter(" ".join(most_words).split()).most_common(int(nt/15))
    most_words = pd.DataFrame(most_words, columns=['Word', 'Frequency'])
    most_words_list = most_words_list + list(most_words['Word'])
    
    tweet_information['text_lenght']= tweet_information['text'].apply(lambda text: len(text))
    tweet_information['tag']=true_text(word)
    tweet_information['built_in_tag']=new_search

    frames.append(tweet_information)
    
most_words_list = Counter(" ".join(most_words_list).split()).most_common(10*nt)
most_words_list = pd.DataFrame(most_words_list, columns=['Word', 'Frequency'])
most_words_list_result=most_words_list.reset_index(drop=True)
most_words_list_result.to_csv('most_words_list_result.csv',index=False)

most_words_list = list(most_words_list['Word'])

# concatinating all 'tweet_information's together:
result = pd.concat(frames)

# resetting the index, and saving:
result=result.reset_index(drop=True)
result.to_csv('tweet_information.csv',index=False)

print('Downloading finished.')
print('Some initial processing has been finished.')

IN covid...
IN fashion...
IN business...
IN travel...
IN technology...
IN science...
IN political...
IN physics...
IN pharmacy...
IN pet...
IN medical...
IN marketing...
IN solar...
IN job...
IN innovation...
IN gym...
IN government...
IN food...
IN engineering...
IN election...
IN culture...
IN bodybuilding...
IN biology...
IN art...
IN architecture...
IN animal...
Some initial processing has been finished.
